In [1]:
import sys
sys.path.insert(1, '../../')
from evaluator import Evaluator
from retriever import *
import pandas as pd
pd.set_option('display.max_colwidth', None)
from transformers import logging
import time
import datetime

logging.set_verbosity_error()



ag_results_w_kpe = pd.read_csv("../../experiments/augmented-generation/results/results_w_kpe-2025-03-14 20:43:58.csv")
ag_results_wo_kpe = pd.read_csv("../../experiments/augmented-generation/results/results_wo_kpe-2025-03-14 20:44:03.csv")


In [2]:
import pandas as pd
from scipy.stats import wilcoxon

def calculate_significance(df_with, df_without, merge_on=['docid', 'query'], metrics=None):
    if metrics is None:
        metrics = ['Vstrict_GPT', 'V_GPT', 'W_GPT', 'A_GPT',
                   'grammar_score', 'succinctness_score', 'readability_score', 'L']
    
    merged_df = pd.merge(df_with, df_without, on=merge_on, suffixes=('_with', '_without'))
    
    results = {}

    for metric in metrics:
        data_with = merged_df[f'{metric}_with']
        data_without = merged_df[f'{metric}_without']
        
        stat, p_value = wilcoxon(data_with, data_without)
        
        results[metric] = {'Wilcoxon_statistic': stat, 'p_value': p_value}
    results_df = pd.DataFrame(results).T
    return results_df


results_df = calculate_significance(ag_results_w_kpe, ag_results_wo_kpe)
results_df


,Wilcoxon_statistic,p_value
Vstrict_GPT,1900.5,0.020654
V_GPT,3284.5,0.032284
W_GPT,3923.0,0.025764
A_GPT,4644.5,0.000328
grammar_score,239.0,0.002872
succinctness_score,1498.5,0.305932
readability_score,559.5,0.002614
L,9285.5,0.000897
